In [2]:
import numpy as np
import torch
import random
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
import matplotlib.pyplot as plt
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:64'

import utils

%load_ext autoreload
%autoreload 2

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Global variable model:

model_name = 'microsoft/Phi-3-mini-4k-instruct' #"openai-community/gpt2-xl" #"meta-llama/Llama-2-7b-hf" #openai-community/gpt2-xl 'microsoft/Phi-3-mini-4k-instruct
dataset_path = 'datasets/ilikecats_20000.csv' #'datasets/(non)animal_2000_gpt-4-turbo.csv'

# # Load model and dataset
# model = utils.LinearFeatureWrapper(model_name, data
model_hf = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype = torch.float16,  trust_remote_code=True)
tokenizer_hf = AutoTokenizer.from_pretrained(model_name)
layers = model_hf.model.layers # Depends on model! Typically model.model.layers. For gpt2: model.transformer.h

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.79s/it]
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
dataset_size = 1024

dataset = utils.Dataset()
dataset.load_data(dataset_path, 'Animal', 'Non-Animal')
pos_dataset, neg_dataset = dataset.get_pos_neg_dataset(dataset_size)

In [5]:
model = utils.LinearFeatureWrapper(model_hf, tokenizer_hf, layers, pos_dataset, neg_dataset)
model.batch_size = 8
model.get_pos_neg_activations(dataset_size = -1)


You are not running the flash-attention implementation, expect numerical differences.


Processing batch 1/128
Processing batch 2/128
Processing batch 3/128
Processing batch 4/128
Processing batch 5/128
Processing batch 6/128
Processing batch 7/128
Processing batch 8/128
Processing batch 9/128
Processing batch 10/128
Processing batch 11/128
Processing batch 12/128
Processing batch 13/128
Processing batch 14/128
Processing batch 15/128
Processing batch 16/128
Processing batch 17/128
Processing batch 18/128
Processing batch 19/128
Processing batch 20/128
Processing batch 21/128
Processing batch 22/128
Processing batch 23/128
Processing batch 24/128
Processing batch 25/128
Processing batch 26/128
Processing batch 27/128
Processing batch 28/128
Processing batch 29/128
Processing batch 30/128
Processing batch 31/128
Processing batch 32/128
Processing batch 33/128
Processing batch 34/128
Processing batch 35/128
Processing batch 36/128
Processing batch 37/128
Processing batch 38/128
Processing batch 39/128
Processing batch 40/128
Processing batch 41/128
Processing batch 42/128
P

In [6]:
test_dataset = utils.Dataset()
test_dataset.load_data('datasets/hc_animal_phi3_inputs.csv', 'Animal', 'Non-Animal')
test_dataset_size = dataset_size
test_dataset_size = 700

test_dataset_1, labels = test_dataset.combine_dataset_get_labels(test_dataset_size)
_ = model.train_and_evaluate_probes(test_dataset_1, labels, max_iter = 20000, use_old_test_dataset=False)
# model.train_and_evaluate_probes(max_iter = 10000)

Processing batch 1/176
Processing batch 2/176
Processing batch 3/176
Processing batch 4/176
Processing batch 5/176
Processing batch 6/176
Processing batch 7/176
Processing batch 8/176
Processing batch 9/176
Processing batch 10/176
Processing batch 11/176
Processing batch 12/176
Processing batch 13/176
Processing batch 14/176
Processing batch 15/176
Processing batch 16/176
Processing batch 17/176
Processing batch 18/176
Processing batch 19/176
Processing batch 20/176
Processing batch 21/176
Processing batch 22/176
Processing batch 23/176
Processing batch 24/176
Processing batch 25/176
Processing batch 26/176
Processing batch 27/176
Processing batch 28/176
Processing batch 29/176
Processing batch 30/176
Processing batch 31/176
Processing batch 32/176
Processing batch 33/176
Processing batch 34/176
Processing batch 35/176
Processing batch 36/176
Processing batch 37/176
Processing batch 38/176
Processing batch 39/176
Processing batch 40/176
Processing batch 41/176
Processing batch 42/176
P

: 